In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision.datasets import ImageFolder
from torchvision import transforms, models

import copy

import random
import os
import numpy as np
import time

from pathlib import Path

def seed_torch(seed=1234):
	random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed) 
	np.random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	torch.cuda.manual_seed_all(seed) # if using multi-GPU.
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True

seed_torch()

root_path = Path.cwd()

tear_data = root_path / "tear_data"
tear_data_pos = tear_data / "pos"
tear_data_neg = tear_data / "neg"

data_transforms = transforms.Compose([
    transforms.Resize((800,800)),
    transforms.ToTensor()
])

tear_dataset = ImageFolder(tear_data,
                      transform = data_transforms
)

tear_dataloader = torch.utils.data.DataLoader(tear_dataset, batch_size=16,
                                             shuffle=True, num_workers=4)

dataset_sizes = len(tear_dataset)
class_names = tear_dataset.classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from PIL import Image

def bina(img, ratio):
    img[img < ratio] = 0
    img[img >= ratio] = 1
    return img
    

def bina_img(batch_img):
    for i, img in enumerate(batch_img):
        img[1] = bina(img[1], 0.12)
        img[2] = bina(img[2], 0.52)
        
        batch_img[i] = img
    return batch_img

def train_model(model, criterion, optimizer, scheduler, num_epochs=5, model_name='resnet'):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        if epoch % 10 == 0:
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

       
        model.train()  # Set model to training mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in tear_dataloader:
            
            inputs = bina_img(inputs)
            
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            scheduler.step()

        epoch_loss = running_loss / dataset_sizes
        epoch_acc = running_corrects.double() / dataset_sizes
        
        if epoch % 10 == 0:
            print(f'Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}',end='\n\n')

        # deep copy the model
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            
            
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print('Best val Acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    
    name_time = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime()) 
    torch.save(best_model_wts, f'./models/{model_name}_{name_time}.pt')
    
    return model

In [2]:
tear_dataset.class_to_idx

{'neg': 0, 'pos': 1}

In [3]:
model_ft = models.resnet18(weights=None)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.to(device)
model_ft = nn.DataParallel(model_ft)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=6, gamma=0.1)
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=20,eta_min=0.05)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=120+1)

Epoch 0/120
----------
Loss: 0.9989 Acc: 0.5917

Epoch 10/120
----------
Loss: 0.4865 Acc: 0.7806

Epoch 20/120
----------
Loss: 0.5261 Acc: 0.7583

Epoch 30/120
----------
Loss: 0.5350 Acc: 0.7444

Epoch 40/120
----------
Loss: 0.5004 Acc: 0.7667

Epoch 50/120
----------
Loss: 0.4168 Acc: 0.8250

Epoch 60/120
----------
Loss: 0.4663 Acc: 0.7833

Epoch 70/120
----------
Loss: 0.3464 Acc: 0.8583

Epoch 80/120
----------
Loss: 0.3154 Acc: 0.8722

Epoch 90/120
----------
Loss: 0.3169 Acc: 0.8917

Epoch 100/120
----------
Loss: 0.2688 Acc: 0.9028

Epoch 110/120
----------
Loss: 0.1945 Acc: 0.9222

Epoch 120/120
----------
Loss: 0.1619 Acc: 0.9444

Training complete in 8m 53s
Best val Acc: 0.950000
